In [1]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

In [2]:
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

train_dataset = datasets.MNIST(root='../dataset/mnist',
                               train=True,
                               download=True,
                               transform=transform
                               )
train_loader = DataLoader(train_dataset, 
                          shuffle=True,
                          batch_size=batch_size)

test_dataset = datasets.MNIST(root='../dataset/mnist',
                              train=False,
                              download=True,
                              transform=transform)
test_loader = DataLoader(test_dataset,
                         shuffle=False,
                         batch_size=batch_size)

100%|██████████| 9912422/9912422 [00:01<00:00, 5615349.56it/s]


Extracting ../dataset/mnist\MNIST\raw\train-images-idx3-ubyte.gz to ../dataset/mnist\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 14354271.10it/s]


Extracting ../dataset/mnist\MNIST\raw\train-labels-idx1-ubyte.gz to ../dataset/mnist\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 5559273.10it/s]


Extracting ../dataset/mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to ../dataset/mnist\MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 4599355.09it/s]

Extracting ../dataset/mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../dataset/mnist\MNIST\raw



In [3]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(784, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        x = self.l5(x) # 最后一层不做激活，crossentropyloss函数里有softmax
        return x

model = Net()


In [4]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


In [5]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print("epoch: {}, loss: {}".format(epoch+1, running_loss/300))
            running_loss = 0.0

In [6]:
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('accuracy on test set: %d %%' % (100 * correct/total))


In [7]:
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

epoch: 1, loss: 2.1974952856699628
epoch: 1, loss: 0.8704013141989708
epoch: 1, loss: 0.4150585024058819
accuracy on test set: 89 %
epoch: 2, loss: 0.3205219047764937
epoch: 2, loss: 0.2648182187477748
epoch: 2, loss: 0.2120881560941537
accuracy on test set: 94 %
epoch: 3, loss: 0.1838088142996033
epoch: 3, loss: 0.16717136508474748
epoch: 3, loss: 0.14853786362335086
accuracy on test set: 95 %
epoch: 4, loss: 0.12671673593421778
epoch: 4, loss: 0.12180363933866223
epoch: 4, loss: 0.11605158546939492
accuracy on test set: 96 %
epoch: 5, loss: 0.09488417473311225
epoch: 5, loss: 0.0941897696070373
epoch: 5, loss: 0.09502469652642806
accuracy on test set: 97 %
epoch: 6, loss: 0.08139359338209033
epoch: 6, loss: 0.07182133807024609
epoch: 6, loss: 0.07180219892257203
accuracy on test set: 96 %
epoch: 7, loss: 0.06122000531914334
epoch: 7, loss: 0.06100650461235394
epoch: 7, loss: 0.06063420043326914
accuracy on test set: 97 %
epoch: 8, loss: 0.05059305114516367
epoch: 8, loss: 0.046845959